In [1]:
import aiohttp
import os
import asyncio
import pandas as pd
import json
import nest_asyncio

# Apply the patch nest_asyncio to allow asynchronous event loops
nest_asyncio.apply()

# We need to reach the JSON file from which we will retrieve the URLs
current_dir = os.getcwd()  # Obtient le répertoire de travail courant
json_path = os.path.join(current_dir, '..', '1_scrap_urls', 'scrap_le_louvre_urls_all_museum.json')

# Lire le fichier JSON
df = pd.read_json(json_path)

# Function to fetch data from a single URL
async def fetch(session, url):
    try:
        async with session.get(url) as response:
            if response.status != 200:
                print(f"Non-200 response for URL: {url} with status: {response.status}")
                return None
            if response.headers.get('Content-Type') != 'application/json':
                print(f"Non-JSON response for URL: {url} with content-type: {response.headers.get('Content-Type')}")
                return None
            return await response.json()
    except aiohttp.ClientError as e:
        print(f"Request failed for URL: {url} with exception: {e}")
        return None
    except aiohttp.ContentTypeError:
        print(f"ContentTypeError for URL: {url}")
        return None
    except aiohttp.ClientPayloadError:
        print(f"ClientPayloadError for URL: {url}")
        return None

# Function to handle retries
async def fetch_with_retry(session, url, retries=3):
    for attempt in range(retries):
        result = await fetch(session, url)
        if result is not None:
            return result
        await asyncio.sleep(2 ** attempt)  # Exponential backoff
    print(f"Failed to fetch URL after {retries} retries: {url}")
    return None

# Function to fetch data from all URLs with retries
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            full_url = f"https://collections.louvre.fr{url}.json"
            tasks.append(fetch_with_retry(session, full_url))
        results = await asyncio.gather(*tasks)
        return results

# Extract the first 25k URLs
urls = df['url'][:25000]

# Execute the requests with an asynchronous way
data = asyncio.run(fetch_all(urls))

# Filter the results to drop None
filtered_data = [d for d in data if d is not None]

# Transform the first 25k JSON API results into 25K entries in a DataFrame
df_final = pd.json_normalize(filtered_data)

# Display the dataframe
df_final


,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index,index.objectType,index.Original artwork,index.People,index.Subjects,index.collection
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,cl020605101,https://collections.louvre.fr/ark:/53355/cl020...,2023-09-15,"Du Roy Francois, deuxiesme de ce nom, soisanti...",None,"[{'type': 'Titre', 'value': 'Du Roy Francois, ...",1567,Réserve Edmond de Rothschild / Recueil : Effig...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,cl020605102,https://collections.louvre.fr/ark:/53355/cl020...,2023-09-15,"Charles, neufiesme de ce no[m] apresent regnant",None,"[{'type': 'Titre', 'value': 'Charles, neufiesm...",1567,Réserve Edmond de Rothschild / Recueil : Effig...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,cl020605103,https://collections.louvre.fr/ark:/53355/cl020...,2023-09-15,"Du Roy Charles neufiesme de ce nom, apresent R...",None,"[{'type': 'Titre', 'value': 'Du Roy Charles ne...",1567,Réserve Edmond de Rothschild / Recueil : Effig...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,cl020605104,https://collections.louvre.fr/ark:/53355/cl020...,2023-09-15,Allégorie de la piété et de la justice,None,"[{'type': 'Titre', 'value': 'Allégorie de la p...",1567,Réserve Edmond de Rothschild / Recueil : Effig...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Load the next 25k urls
urls = df['url'][25000:50000]

# Execute the asynchronous fetch of the next 25k urls
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate the next 25k results with the existing DataFrame containing our entries so far
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame with the concatenated results
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,cl020577623,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,Fille de qualité en habit d'été,None,"[{'type': 'Titre', 'value': 'Fille de qualité ...",,Réserve Edmond de Rothschild / Recueil : Portr...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
49996,cl020577624,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,Femme de qualité en habit d'été,None,"[{'type': 'Titre', 'value': 'Femme de qualité ...",,Réserve Edmond de Rothschild / Recueil : Portr...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
49997,cl020577625,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,Femme de qualité en déshabillé d'été,None,"[{'type': 'Titre', 'value': 'Femme de qualité ...",,Réserve Edmond de Rothschild / Recueil : Portr...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
49998,cl020577626,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,Fille de qualité en habit d'été à la Vestale,None,"[{'type': 'Titre', 'value': 'Fille de qualité ...",,Réserve Edmond de Rothschild / Recueil : Portr...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [3]:
# Load URLs from dataframe
urls = df['url'][50000:75000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,cl020551556,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-20,Jeux des Petits Polissons de Paris,None,"[{'type': 'Titre', 'value': 'Jeux des Petits P...",,Réserve Edmond de Rothschild,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
74996,cl020551557,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-20,Jeux des Petits Polissons de Paris,None,"[{'type': 'Titre', 'value': 'Jeux des Petits P...",,Réserve Edmond de Rothschild,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
74997,cl020551558,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-20,Jeux des Petits Polissons de Paris,None,"[{'type': 'Titre', 'value': 'Jeux des Petits P...",,Réserve Edmond de Rothschild,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
74998,cl020551559,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-20,Jeux des Petits Polissons de Paris,None,"[{'type': 'Titre', 'value': 'Jeux des Petits P...",,Réserve Edmond de Rothschild,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [4]:
# Load URLs from dataframe
urls = df['url'][75000:100000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020538978.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020538982.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020538979.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020538980.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020538985.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request fa

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,cl020526450,https://collections.louvre.fr/ark:/53355/cl020...,,Monsieur Sériziat,None,"[{'type': 'Titre', 'value': 'Monsieur Sériziat'}]",,Atelier de la chalcographie,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
99996,cl020526451,https://collections.louvre.fr/ark:/53355/cl020...,,Madame Sériziat,Portrait de,"[{'type': 'Titre', 'value': 'Madame Sériziat/P...",,Atelier de la chalcographie,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
99997,cl020526452,https://collections.louvre.fr/ark:/53355/cl020...,,La leçon de clavecin,None,"[{'type': 'Titre', 'value': 'La leçon de clave...",,Atelier de la chalcographie,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
99998,cl020526453,https://collections.louvre.fr/ark:/53355/cl020...,2023-02-22,Figure d'homme nu.,None,"[{'type': 'Titre', 'value': 'Figure d'homme nu...",,Atelier de la chalcographie,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Load URLs from dataframe
urls = df['url'][100000:125000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124995,cl020500287,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Esquisse d'une femme nue, de dos",None,"[{'type': 'Titre', 'value': 'Esquisse d'une fe...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'Saint-Morys'}],NaN,NaN,NaN
124996,cl020500288,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,Etude d'une décoration pour un vase,None,"[{'type': 'Titre', 'value': 'Etude d'une décor...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Amateur A (Lugt)'}, {'value': 'Sai...",NaN,NaN,NaN
124997,cl020500289,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,Fragment de grappe de raisin,None,"[{'type': 'Titre', 'value': 'Fragment de grapp...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Amateur A (Lugt)'}, {'value': 'Sai...",NaN,NaN,NaN
124998,cl020500290,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Un homme agenouillé, de dos, tourné vers la dr...",None,"[{'type': 'Titre', 'value': 'Un homme agenouil...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Load URLs from dataframe
urls = df['url'][125000:150000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020217273.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020217439.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020217437.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020217436.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020217435.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request fa

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,cl020208923,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Enfant nu, assis, de dos, regardant vers le haut",None,"[{'type': 'Titre', 'value': 'Enfant nu, assis,...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149996,cl020208924,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Enfant nu, appuyé sur le bras droit, et levant...",None,"[{'type': 'Titre', 'value': 'Enfant nu, appuyé...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149997,cl020208925,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Etudes d'enfants, de mains, d'une draprie et d...",None,"[{'type': 'Titre', 'value': 'Etudes d'enfants,...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149998,cl020208926,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,Tête de femme et enfant nu,None,"[{'type': 'Titre', 'value': 'Tête de femme et ...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Jésus-Christ'}, {'value': 'Vierge ...","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN


In [7]:
# Load URLs from dataframe
urls = df['url'][150000:175000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020138025.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020138023.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020138061.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020138059.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request failed for URL: https://collections.louvre.fr/en/ark:/53355/cl020138054.json with exception: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte
Request fa

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174995,cl020130053,https://collections.louvre.fr/ark:/53355/cl020...,2023-07-05,Hommes travaillant sur une langue de sable déc...,None,"[{'type': 'Titre', 'value': 'Hommes travaillan...",,Réserve des grands albums / Album Boudin Eugèn...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174996,cl020130054,https://collections.louvre.fr/ark:/53355/cl020...,2023-07-05,Barque échouée à marée basse,None,"[{'type': 'Titre', 'value': 'Barque échouée à ...",,Réserve des grands albums / Album Boudin Eugèn...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174997,cl020130055,https://collections.louvre.fr/ark:/53355/cl020...,2023-07-05,Trois canots sur le sable,None,"[{'type': 'Titre', 'value': 'Trois canots sur ...",,Réserve des grands albums / Album Boudin Eugèn...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174998,cl020130056,https://collections.louvre.fr/ark:/53355/cl020...,2023-07-05,Barque à voiles,None,"[{'type': 'Titre', 'value': 'Barque à voiles'}]",,Réserve des grands albums / Album Boudin Eugèn...,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Load URLs from dataframe
urls = df['url'][175000:200000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,cl020104993,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Etude d'une tête d'homme, vue de face et croqu...",None,"[{'type': 'Titre', 'value': 'Etude d'une tête ...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'Saint-Morys'}],NaN,NaN,NaN
199996,cl020104994,https://collections.louvre.fr/ark:/53355/cl020...,2023-11-13,"Tête de jeune fille, vue de trois quarts vers ...",None,"[{'type': 'Titre', 'value': 'Tête de jeune fil...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Dezallier d'Argenville, Antoine-Jo...",NaN,NaN,NaN
199997,cl020104995,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,La transfiguration du Christ et la guérison du...,None,"[{'type': 'Titre', 'value': 'La transfiguratio...",,Grand format,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Orsay, Pierre Marie Gaspard Grimod...","[{'value': 'apôtre'}, {'value': 'Jésus-Christ'}]","[{'value': 'Transfiguration'}, {'value': 'ICON...",NaN
199998,cl020104996,https://collections.louvre.fr/ark:/53355/cl020...,2023-11-20,Saint François de Paule traversant le détroit ...,None,"[{'type': 'Titre', 'value': 'Saint François de...",,Petit format,,False,...,NaN,[{'value': 'Sicile'}],NaN,NaN,NaN,NaN,"[{'value': 'Orsay, Pierre Marie Gaspard Grimod...","[{'value': 'François de Paule, saint'}]","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN


In [9]:
# Load URLs from dataframe
urls = df['url'][200000:225000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Description/Features,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'cheval'}, {'value': 'animal'}]","[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224995,cl020023815,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-03,Notes manuscrites,None,"[{'type': 'Titre', 'value': 'Notes manuscrites'}]",,Réserve des petits albums / Album Neuville Alp...,,False,...,NaN,[{'value': 'Arras'}],NaN,NaN,NaN,NaN,"[{'value': 'Moreau-Nélaton, Etienne'}]",NaN,NaN,NaN
224996,cl020023816,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-03,Croquis d'une maison,None,"[{'type': 'Titre', 'value': 'Croquis d'une mai...",,Réserve des petits albums / Album Neuville Alp...,,False,...,NaN,[{'value': 'Villersexel'}],NaN,NaN,NaN,NaN,"[{'value': 'Moreau-Nélaton, Etienne'}]",NaN,NaN,NaN
224997,cl020023817,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-03,Notes manuscrites,None,"[{'type': 'Titre', 'value': 'Notes manuscrites'}]",,Réserve des petits albums / Album Neuville Alp...,,False,...,NaN,[{'value': 'Villersexel'}],NaN,NaN,NaN,NaN,"[{'value': 'Moreau-Nélaton, Etienne'}]",NaN,NaN,NaN
224998,cl020023818,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-03,Croquis,None,"[{'type': 'Titre', 'value': 'Croquis'}]",,Réserve des petits albums / Album Neuville Alp...,,False,...,NaN,[{'value': 'Villersexel'}],NaN,NaN,NaN,NaN,"[{'value': 'Moreau-Nélaton, Etienne'}]",NaN,NaN,NaN


In [10]:
# Load URLs from dataframe
urls = df['url'][225000:250000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Places,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'Égypte'}, {'value': 'Médamoud'}, {...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,"[{'value': 'de Thèbes à Assouan'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'Égypte'}, {'value': 'Fayoum'}]","[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,cl010496141,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,coupe,,"[{'type': 'Dénomination', 'value': 'coupe (fra...",Epoque / période : classique ; figures rouges\...,non exposé,,False,...,"[{'value': 'Grèce'}, {'value': 'Attique'}, {'v...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249996,cl010496161,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-17,Camée : Vénus et l'amour,,"[{'type': 'Titre', 'value': 'Camée : Vénus et ...",Epoque / période : époque moderne = Ancien Rég...,"Denon, [OArt] Salle 705 - Galerie d'Apollon, V...","Salle 705, Aile Denon, Niveau 1",False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'gemme et pierre dure'}, {'value': ..."
249997,cl010496165,https://collections.louvre.fr/ark:/53355/cl010...,2023-12-20,Fragments de plat,,"[{'type': 'Titre', 'value': 'Fragments de plat...",Date de création/fabrication : 1er quart du XI...,non exposé,,False,...,"[{'value': 'Proche-Orient arabe'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249998,cl010496166,https://collections.louvre.fr/ark:/53355/cl010...,2023-12-20,Fragments de plat,,"[{'type': 'Titre', 'value': 'Fragments de plat...",Date de création/fabrication : 1er quart du XI...,non exposé,,False,...,"[{'value': 'Proche-Orient arabe'}, {'value': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Load URLs from dataframe
urls = df['url'][250000:275000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274995,cl010402096,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-11,"objet indéterminé, fragment",,"[{'type': 'Dénomination', 'value': 'objet indé...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274996,cl010402097,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-11,"fiche, fragment ; clou, fragment",,"[{'type': 'Dénomination', 'value': 'fiche, fra...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274997,cl010402098,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-11,"élément de construction, fragment",,"[{'type': 'Dénomination', 'value': 'élément de...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274998,cl010402099,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-11,"élément de construction, fragment",,"[{'type': 'Dénomination', 'value': 'élément de...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Load URLs from dataframe
urls = df['url'][275000:300000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,cl010349507,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,coupe ; assiette ; plat,,"[{'type': 'Dénomination', 'value': 'coupe ; as...",Date de création/fabrication : 1e quart IIIe s...,"Nantes (France), Musée Dobrée",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299996,cl010349508,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,coupe ; assiette ; plat,,"[{'type': 'Dénomination', 'value': 'coupe ; as...",Date de création/fabrication : IIIe s. av. J.-...,"Nantes (France), Musée Dobrée",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299997,cl010349509,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,coupe ; assiette ; plat,,"[{'type': 'Dénomination', 'value': 'coupe ; as...",Date de création/fabrication : IVe s. av. J.-C...,"Lyon (France), Musée des Beaux-Arts",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299998,cl010349512,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,coupe ; fragment,,"[{'type': 'Dénomination', 'value': 'coupe ; fr...",Date de création/fabrication : Ie s. av. J.-C....,"Saint-Germain-en-Laye (France), Musée d'archéo...",,False,...,"[{'value': 'contenu du texte'}, {'value': 'mar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Load URLs from dataframe
urls = df['url'][300000:325000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324995,cl010308370,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-02,flacon,,"[{'type': 'Dénomination', 'value': 'flacon'}]",Epoque / période : antiquité tardive,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324996,cl010308371,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-02,bouteille,,"[{'type': 'Dénomination', 'value': 'bouteille'}]",Epoque / période : romain impérial,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324997,cl010308372,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-02,bouteille,,"[{'type': 'Dénomination', 'value': 'bouteille'}]",Epoque / période : islamique,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324998,cl010308373,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-02,fragment indéterminé,,"[{'type': 'Dénomination', 'value': 'fragment i...",Epoque / période : médiéval,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Load URLs from dataframe
urls = df['url'][325000:350000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349995,cl010270790,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-18,Tirage intégral de la main gauche d'un couros.,,"[{'type': 'Dénomination', 'value': 'statue (fr...",Epoque / période : moderne\r\n\r\nDate de créa...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'lieux'}, {'value': 'Paris musée du..."
349996,cl010270791,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-18,Tirage d'un fragment de tête masculine,,"[{'type': 'Titre', 'value': 'Tirage d'un fragm...",Epoque / période : moderne,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Autres oeuvres'}, {'value': 'origi..."
349997,cl010270792,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-27,Tirage d'une plaque représentant un Grec contr...,,"[{'type': 'Dénomination', 'value': 'plaque de ...",Epoque / période : moderne,"Ecuries du château de Versailles, [AGER] Petit...",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Monuments'}, {'value': 'Halicarnas..."
349998,cl010270793,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-27,Tirage d'une plaque représentant deux Grecs co...,,"[{'type': 'Dénomination', 'value': 'plaque de ...",Epoque / période : moderne,"Ecuries du château de Versailles, [AGER] Petit...",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Monuments'}, {'value': 'Halicarnas..."


In [15]:
# Load URLs from dataframe
urls = df['url'][350000:375000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374995,cl010216140,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,estèque ; tranchet,,"[{'type': 'Dénomination', 'value': 'estèque (?...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374996,cl010216141,https://collections.louvre.fr/ark:/53355/cl010...,2023-12-03,étendard,,"[{'type': 'Dénomination', 'value': 'étendard (...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374997,cl010216142,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,estèque ; tranchet,,"[{'type': 'Dénomination', 'value': 'estèque (?...",,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374998,cl010216143,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,étendard,,"[{'type': 'Dénomination', 'value': 'étendard (...",,"Strasbourg (France), Bibliothèque nationale et...",,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Load URLs from dataframe
urls = df['url'][375000:400000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,cl010168145,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,tablette,,"[{'type': 'Dénomination', 'value': 'tablette'}]",Date de création/fabrication : Paléo-assyrien ...,non exposé,,False,...,"[{'value': 'inscription'}, {'value': 'texte éc...","[{'value': 'écriture'}, {'value': 'type d'écri...",NaN,"[{'value': 'langue'}, {'value': 'langue chamit...",NaN,NaN,NaN,NaN,NaN,NaN
399996,cl010168146,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,tablette,,"[{'type': 'Dénomination', 'value': 'tablette'}]",Date de création/fabrication : Paléo-assyrien ...,non exposé,,False,...,"[{'value': 'inscription'}, {'value': 'correspo...","[{'value': 'écriture'}, {'value': 'type d'écri...",NaN,"[{'value': 'langue'}, {'value': 'langue chamit...",NaN,NaN,NaN,NaN,NaN,NaN
399997,cl010168147,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,tablette,,"[{'type': 'Dénomination', 'value': 'tablette'}]",Date de création/fabrication : Paléo-assyrien ...,non exposé,,False,...,"[{'value': 'inscription'}, {'value': 'texte éc...","[{'value': 'écriture'}, {'value': 'type d'écri...",NaN,"[{'value': 'langue'}, {'value': 'langue chamit...",NaN,NaN,NaN,NaN,NaN,NaN
399998,cl010168148,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,tablette,,"[{'type': 'Dénomination', 'value': 'tablette'}]",Date de création/fabrication : Paléo-assyrien ...,non exposé,,False,...,"[{'value': 'inscription'}, {'value': 'texte éc...","[{'value': 'écriture'}, {'value': 'type d'écri...",NaN,"[{'value': 'langue'}, {'value': 'langue chamit...",NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Load URLs from dataframe
urls = df['url'][400000:425000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

Non-200 response for URL: https://collections.louvre.fr/en/ark:/53355/cl010139032.json with status: 404
Non-200 response for URL: https://collections.louvre.fr/en/ark:/53355/cl010139032.json with status: 404
Non-200 response for URL: https://collections.louvre.fr/en/ark:/53355/cl010139032.json with status: 404
Failed to fetch URL after 3 retries: https://collections.louvre.fr/en/ark:/53355/cl010139032.json


,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424994,cl010127074,https://collections.louvre.fr/ark:/53355/cl010...,2024-05-02,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : Dynastie des Su...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424995,cl010127075,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-27,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : Dynastie des Su...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424996,cl010127076,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-27,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : Dynastie des Su...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424997,cl010127077,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-27,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : Dynastie des Su...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Load URLs from dataframe
urls = df['url'][425000:450000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449994,cl010094062,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-04,Jeune femme ailée tenant une lyre,,"[{'type': 'Titre', 'value': 'Jeune femme ailée...",Date de création/fabrication : 1800 - 1900,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'esquisse'}],NaN
449995,cl010094064,https://collections.louvre.fr/ark:/53355/cl010...,2023-10-20,Barbet de Jouy,,"[{'type': 'Titre', 'value': 'Barbet de Jouy'}]",Date de création/fabrication : 1800 - 2000,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'buste'}],NaN
449996,cl010094034,https://collections.louvre.fr/ark:/53355/cl010...,2024-04-16,Tête de femme aux yeux clos,,"[{'type': 'Titre', 'value': 'Tête de femme aux...",Date de création/fabrication : 1800 - 2000 (st...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'tête'}],NaN
449997,cl010094035,https://collections.louvre.fr/ark:/53355/cl010...,2023-12-08,Tête de Christ couronné d'épines,,"[{'type': 'Titre', 'value': 'Tête de Christ co...",Date de création/fabrication : 1485 - 1500,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'value': 'fragment de statue'}],NaN


In [19]:
# Load URLs from dataframe
urls = df['url'][450000:475000]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474994,cl010043864,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,lampe ovale allongée à canal,,"[{'type': 'Dénomination', 'value': 'lampe oval...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474995,cl010043866,https://collections.louvre.fr/ark:/53355/cl010...,2024-03-19,lampe,,"[{'type': 'Dénomination', 'value': 'lampe'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474996,cl010043867,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,lampe théière,,"[{'type': 'Dénomination', 'value': 'lampe théi...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474997,cl010043869,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,lampe ; simulacre,,"[{'type': 'Dénomination', 'value': 'lampe ; si...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Load URLs from dataframe
urls = df['url'][475000:]

# Execute the asynchronous fetch
data = asyncio.run(fetch_all(urls))

# Filter out None responses
data = [item for item in data if item is not None]

# Normalize JSON data into a DataFrame
df_final_increment = pd.json_normalize(data)

# Concatenate with the existing DataFrame
df_final = pd.concat([df_final, df_final_increment], ignore_index=True)

# Display the final DataFrame
df_final

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506242,cl020009356,https://collections.louvre.fr/ark:/53355/cl020...,2023-11-13,L'ascension d'Elie dans un char de feu devant ...,None,"[{'type': 'Titre', 'value': 'L'ascension d'Eli...",,Petit format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Destouches, chevalier'}, {'value':...","[{'value': 'Elie'}, {'value': 'Elisée, prophèt...","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN
506243,cl020009357,https://collections.louvre.fr/ark:/53355/cl020...,2023-10-25,Annonciation,None,"[{'type': 'Titre', 'value': 'Annonciation'}]",,Petit format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Hudson, Thomas'}, {'value': 'Saint...","[{'value': 'Gabriel, archange'}, {'value': 'Vi...","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN
506244,cl020009358,https://collections.louvre.fr/ark:/53355/cl020...,2024-01-12,L'Adoration des bergers,None,"[{'type': 'Titre', 'value': 'L'Adoration des b...",,Petit format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Nourri'}, {'value': 'Saint-Morys'}]","[{'value': 'Joseph, saint'}, {'value': 'Vierge...","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN
506245,cl020009359,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,Vierge et l'Enfant entourés d'anges,None,"[{'type': 'Titre', 'value': 'Vierge et l'Enfan...",,Petit format,,False,...,NaN,NaN,NaN,NaN,NaN,"[{'value': 'Jésus-Christ'}, {'value': 'Vierge ...","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN


In [21]:
# See if there are some duplicates
df_final["arkId"].value_counts()

cl020009209    2
cl020009202    2
cl020009216    2
cl020009341    2
cl020009217    2
              ..
cl020136393    1
cl020136392    1
cl020136391    1
cl020136390    1
cl020009380    1
Name: arkId, Length: 506207, dtype: int64

In [22]:
# Withdraw duplicates
df_final_without_duplicates = df_final.loc[df_final.astype(str).drop_duplicates().index]
df_final_without_duplicates

,arkId,url,modified,title,titleComplement,denominationTitle,displayDateCreated,currentLocation,room,isMuseesNationauxRecuperation,...,index.Nature of text,index.Script,index.Names and titles,index.Language,index.collection,index.People,index.Subjects,index,index.objectType,index.Original artwork
0,cl010000006,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-20,moule d'orfèvre ; boucle d'oreille,,"[{'type': 'Dénomination', 'value': 'moule d'or...",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cl010000007,https://collections.louvre.fr/ark:/53355/cl010...,2024-02-16,figurine,,"[{'type': 'Dénomination', 'value': 'figurine'}]",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cl010000008,https://collections.louvre.fr/ark:/53355/cl010...,2023-11-13,verrou,,"[{'type': 'Dénomination', 'value': 'verrou'}]",Date de création/fabrication : époque islamiqu...,non exposé,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cl010000010,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-19,papyrus documentaire,,"[{'type': 'Dénomination', 'value': 'papyrus do...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,"[{'value': 'document fiscal'}, {'value': 'ordr...","[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cl010000011,https://collections.louvre.fr/ark:/53355/cl010...,2024-01-25,papyrus,,"[{'type': 'Dénomination', 'value': 'papyrus (f...",Date de création/fabrication : époque byzantin...,non exposé,,False,...,NaN,"[{'value': 'grec'}, {'value': 'grec cursif'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506222,cl020009379,https://collections.louvre.fr/ark:/53355/cl020...,2024-02-14,Le Christ portant la Croix dans un encadrement...,None,"[{'type': 'Titre', 'value': 'Le Christ portant...",,Petit format,,False,...,NaN,NaN,NaN,NaN,[{'value': 'Saint-Morys'}],[{'value': 'Jésus-Christ'}],"[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN
506224,cl020009258,https://collections.louvre.fr/ark:/53355/cl020...,2023-03-01,"Cinq hommes debout, se dirigeant vers la droit...",None,"[{'type': 'Titre', 'value': 'Cinq hommes debou...",,Petit format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Lenglier, P.'}, {'value': 'Saint-M...","[{'value': 'apôtre'}, {'value': 'Jésus-Christ'}]","[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN
506225,cl020009259,https://collections.louvre.fr/ark:/53355/cl020...,2023-05-15,Ruines avec la pyramide de Cestius,None,"[{'type': 'Titre', 'value': 'Ruines avec la py...",,Petit format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Cabinet du Roi'}, {'value': 'Marie...",[{'value': 'Cestius'}],NaN,NaN,NaN,NaN
506226,cl020009260,https://collections.louvre.fr/ark:/53355/cl020...,2023-11-13,Prédication d'un apôtre au milieu de ruines,None,"[{'type': 'Titre', 'value': 'Prédication d'un ...",,Grand format,,False,...,NaN,NaN,NaN,NaN,"[{'value': 'Cabinet du Roi'}, {'value': 'Marie...",[{'value': 'apôtre'}],"[{'value': 'ICONOGRAPHIE RELIGIEUSE'}, {'value...",NaN,NaN,NaN


In [22]:
# Save the dataframe with all the Le Louvre artworks and corresponding information into a csv file
df_final_without_duplicates.to_csv("all_works_of_art_le_louvre_uncleaned.csv")